<a href="https://colab.research.google.com/github/AbeHandler/AbeHandler.github.io/blob/master/EricSaeedDataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import json
import enum

with open('all.jsonl', 'r') as f:
  lines = f.readlines()
  lines = [o.strip("\n").replace('"', "'").replace("'", '"').strip('"') for o in lines]

data = [json.loads(line) for line in lines]
df = pd.DataFrame(data)
df = df[df["number"].apply(lambda x: len(x) > 0)]

participants = df.groupby("guidString").count().reset_index()[["guidString", "turn"]]

ids = participants[participants["turn"] > 175]["guidString"].to_list()
ids

['0bb3193c-b54b-4b70-b19b-952ee242638a',
 '0deac4b8-8f85-4083-aa82-7df611879230',
 '0f9eddbc-54a7-4be3-b581-60475c09f961',
 '1478084c-c08c-4295-be9a-1665da41b2f0',
 '354d7416-7fe6-4924-99fb-c3c0a36737b1',
 '40326c10-4e4f-49ae-a596-73febb59d724',
 '4416d72c-5d11-40c4-9d1c-6fcf8d8a4533',
 '45d5d3d7-b860-43a6-b197-6ef68525289b',
 '83caf53c-9308-4cc3-a63f-f9929c8c13a6',
 '93d83659-06af-4363-a578-55fe62b453fd',
 'cbe3c843-bf3d-4b48-9479-8a855f209c55',
 'd03a7391-4d1d-4a21-b36e-205e71320beb',
 'efbabaa4-7895-4d32-b844-757a8877716a']

In [3]:
df["CamZ"].apply(lambda x: math.floor(float(x))).unique()

array([ 0, -5,  5, -1])

In [19]:
import pandas as pd
import json
import enum
import math

id2clevelandanalysis = {}

with open('data.jsonl', 'r') as f:
    lines = f.readlines()
    lines = [o.strip("\n").replace('"', "'").replace("'", '"').strip('"') for o in lines]
    data = [json.loads(line) for line in lines]


df = pd.DataFrame(data)
df = df[df["number"].apply(lambda x: len(x) > 0)]

print(len(df))
df = df[df["guidString"].apply(lambda x: x in ids)]
print(len(df))

df["CamZ"] = df["CamZ"].apply(lambda x: round(float(x)))
df["CamZ"] = df["CamZ"].apply(lambda x: -5 if x <= -5 else x) # if more neg then -5, just clip to -5
df["CamZ"] = df["CamZ"].apply(lambda x: 5 if x >= 5 else x) # if more pos then +5, just clip to +5
df["CamZ"] = df["CamZ"].apply(lambda x: 0 if abs(int(x)) <= 1 else x) # if more pos then +5, just clip to +5


assert len(df["CamZ"].apply(lambda x: math.floor(float(x))).unique()) == 3

class Position(enum.Enum):
    RIGHT = 5
    CENTER =  0
    LEFT = -5

df["position"] = df["CamZ"].apply(lambda x: Position(round(float(x))).name)

df["number"] = df["number"].apply(lambda x: float(x))
df["Lvar"] = df["Lvar"].apply(lambda x: float(x))
df["Rvar"] = df["Rvar"].apply(lambda x: float(x))
df["Cleveland"] = df.index > 90
df["larger"] = df[["Lvar", "Rvar"]].max(axis=1)
df["smaller"] = df[["Lvar", "Rvar"]].min(axis=1)
df["data_pct"] = (df["smaller"] / df["larger"]) * 100
df = df.drop("larger", axis=1)
df = df.drop("smaller", axis=1)


# cleveland analysis
dfC = df[df["Cleveland"]].copy()
dfC["bias"] = dfC["number"] - dfC["data_pct"]
dfC['right_bigger'] = (dfC['Rvar'] > dfC['Lvar'])
dfC['sideSmall'] = (dfC['right_bigger'] & (dfC['position'] == 'LEFT')) | (~dfC['right_bigger'] & (dfC['position'] == 'RIGHT'))
dfC = dfC[dfC["position"] != "CENTER"]
dfC = dfC.groupby('sideSmall').agg({'bias': ['mean', 'std']}).reset_index()

dfnC = df[~df["Cleveland"]].copy()
dfnC["number"] = dfnC["number"].apply(lambda x: int(x))
dfnC['Lvar'] = dfnC['Lvar'].apply(lambda x: x * 100)
dfnC["ncbias"] = dfnC["number"] - dfnC['Lvar']
dfnC["error"] = dfnC["number"] - dfnC['Lvar']
dfnC["error"] = dfnC["error"].abs()
dfnC = dfnC.groupby('position')['error'].mean()


1615
1443


In [22]:
dfC = df[df["Cleveland"]].copy()
dfC["bias"] = dfC["number"] - dfC["data_pct"]
dfC['right_bigger'] = (dfC['Rvar'] > dfC['Lvar'])
dfC['sideSmall'] = (dfC['right_bigger'] & (dfC['position'] == 'LEFT')) | (~dfC['right_bigger'] & (dfC['position'] == 'RIGHT'))
dfC = dfC[dfC["position"] != "CENTER"]
dfC.groupby(['sideSmall', "guidString"]).agg({'bias': ['mean', 'std']}).reset_index().sort_values("guidString")

#dfC.groupby(['sideSmall']).agg({'bias': ['mean', 'std']}).reset_index()

sideSmall                            guidString       bias           
                                                         mean        std
0      False  0bb3193c-b54b-4b70-b19b-952ee242638a -16.290757  24.903898
8       True  0bb3193c-b54b-4b70-b19b-952ee242638a -16.739775  30.939423
1      False  0deac4b8-8f85-4083-aa82-7df611879230 -13.987487  19.379505
9       True  0deac4b8-8f85-4083-aa82-7df611879230 -10.156133  23.093033
2      False  40326c10-4e4f-49ae-a596-73febb59d724 -31.872901  34.087705
10      True  40326c10-4e4f-49ae-a596-73febb59d724 -34.959772  36.657262
3      False  45d5d3d7-b860-43a6-b197-6ef68525289b -20.354889  23.610503
11      True  45d5d3d7-b860-43a6-b197-6ef68525289b -11.332697  29.291265
4      False  83caf53c-9308-4cc3-a63f-f9929c8c13a6 -18.253148  24.591729
12      True  83caf53c-9308-4cc3-a63f-f9929c8c13a6 -11.730184  33.164833
5      False  93d83659-06af-4363-a578-55fe62b453fd -15.897424  24.512806
13      True  93d83659-06af-4363-a578-55fe62b453fd  -2.705921  32.438457
6      False  cbe3c843-bf3d-4b48-9479-8a855f209c55 -16.075568  23.103299
14      True  cbe3c843-bf3d-4b48-9479-8a855f209c55  -7.203858  29.216472
7      False  d03a7391-4d1d-4a21-b36e-205e71320beb -21.392352  29.483465
15      True  d03a7391-4d1d-4a21-b36e-205e71320beb -22.490906  33.345477

In [23]:
from scipy.stats import ttest_ind
group_true = dfC[dfC['sideSmall'] == True]['bias']
group_false = dfC[dfC['sideSmall'] == False]['bias']

# Performing an independent t-test
t_stat, p_value = ttest_ind(group_true, group_false)


t_stat, p_value

(2.5668411268929616, 0.010423962758743953)

In [21]:
dfC

sideSmall       bias           
                  mean        std
0     False -19.150771  25.996619
1      True -14.112012  32.419034